#### Code A

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import shutil
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from IPython.display import clear_output  # 실시간 그래프 갱신
import os

# 디바이스 설정 (GPU 사용 가능 여부 확인)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to the folder you want to delete
folder_path = "checkpoints/train/weights"

if os.path.exists(folder_path):
    # 폴더 내의 모든 파일과 하위 폴더를 삭제
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            # 파일이나 심볼릭 링크인 경우 삭제
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
                print(f"파일 '{file_path}'이 삭제되었습니다.")
            # 디렉토리인 경우 삭제
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
                print(f"폴더 '{file_path}'이 삭제되었습니다.")
        except Exception as e:
            print(f"'{file_path}'를 삭제하는 중 에러가 발생했습니다: {e}")
else:
    print(f"폴더 '{folder_path}'가 존재하지 않습니다.")

#### Code B

In [ ]:
# 하이퍼파라미터 설정
EPOCH = 20        # 총 학습 에포크 수
pre_epoch = 0      # 시작 에포크 (재학습 시 사용)
BATCH_SIZE = 16    # 배치 크기
LR = 0.01          # 학습률
TRAIN_DATA_PERCENT = 0.8  # 훈련 데이터 비율

#### Code C

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.data = self._load_data()

    def _load_data(self):
        data = []
        for cls in self.classes:
            class_path = os.path.join(self.root_dir, cls)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                label = self.class_to_idx[cls]
                data.append((img_path, label))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 전체 데이터셋 로드
dataset = CustomDataset(root_dir="data", transform=transform)

# 훈련과 테스트 비율 설정 (80% 훈련, 20% 테스트)
train_size = int(TRAIN_DATA_PERCENT * len(dataset))
test_size = len(dataset) - train_size

# 랜덤하게 나누기
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"훈련 데이터 수: {len(train_dataset)}, 테스트 데이터 수: {len(test_dataset)}")

#### Code D

In [ ]:
# 클래스 수 확인
classes = dataset.classes  # 원본 데이터셋에서 클래스 정보 추출
NUM_CLASSES = len(classes)

# ResNet-18 모델 불러오기 및 수정
net = models.resnet18()  # 사전 학습된 가중치를 사용하지 않음
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, NUM_CLASSES)  # 출력 차원을 클래스 수로 변경
net = net.to(device)  # 모델을 디바이스로 이동

#### Code E

In [ ]:
# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

#### Code F

In [ ]:
# 훈련 루프 시작
# 훈련 및 테스트 결과를 기록할 리스트
train_losses = []
train_accuracies = []
test_accuracies = []

# 훈련 루프 시작
for epoch in range(EPOCH):
    print(f'\n에폭: {epoch + 1}')
    net.train()
    sum_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader, 0):
        # 데이터 준비
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward 및 Backward 연산
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 계산
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).sum().item()

    # 에폭당 훈련 손실 및 정확도 기록
    train_losses.append(sum_loss / len(train_loader))
    train_accuracies.append(100. * correct / total)

    # 모델 저장
    model_scripted = torch.jit.script(net)
    model_scripted.save(os.path.join('checkpoints/train/weights', f'resnet_epoch_{epoch + 1}.pt'))

    # 테스트 데이터로 정확도 측정
    print('테스트 진행 중...')
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += predicted.eq(labels.data).sum().item()

    test_accuracy = 100. * correct / total
    test_accuracies.append(test_accuracy)

    # 그래프 실시간 출력
    clear_output(wait=True)  # 이전 출력 지우기
    plt.figure(figsize=(10, 5))

    # 훈련 손실 그래프
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epoch + 2), train_losses, marker='o', label='Train Loss')
    plt.title('Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # 훈련 및 테스트 정확도 그래프
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epoch + 2), train_accuracies, marker='o', label='Train Accuracy')
    plt.plot(range(1, epoch + 2), test_accuracies, marker='o', label='Test Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # 현재 정확도 출력
    print(f'[에폭 {epoch + 1}] 훈련 손실: {train_losses[-1]:.3f} | '
          f'훈련 정확도: {train_accuracies[-1]:.3f}% | '
          f'테스트 정확도: {test_accuracies[-1]:.3f}%')

print(f'훈련이 완료되었습니다. 총 에포크 수는 {EPOCH}입니다.')